# SQL Tasks Tier 2

### First Part using PHPMyAdmin

    /* Welcome to the SQL mini project. You will carry out this project partly in
    the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

    This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
    your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
    you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

    Otherwise, the questions in the case study are exactly the same as with Tier 1. 

    PART 1: PHPMyAdmin
    You will complete questions 1-9 below in the PHPMyAdmin interface. 
    Log in by pasting the following URL into your browser, and
    using the following Username and Password:

    URL: https://sql.springboard.com/
    Username: student
    Password: learn_sql@springboard

    The data you need is in the "country_club" database. This database
    contains 3 tables:
        i) the "Bookings" table,
        ii) the "Facilities" table, and
        iii) the "Members" table.

    In this case study, you'll be asked a series of questions. You can
    solve them using the platform, but for the final deliverable,
    paste the code for each solution into this script, and upload it
    to your GitHub.

    Before starting with the questions, feel free to take your time,
    exploring the data, and getting acquainted with the 3 tables. */

    /* Table columns -
    Bookings - bookid, facid, memid, starttime, slots
    Facilities - facid, name, membercost, guestcost, initialoutlay, monthlymaintenance
    Members - memid, surname, firstname, address, zipcode, telephone, recommendedby, joindate
    */

    /* QUESTIONS 
    /* Q1: Some of the facilities charge a fee to members, but some do not.
    Write a SQL query to produce a list of the names of the facilities that do. */

    SELECT *
    FROM Facilities
    WHERE membercost > 0

    /* Q2: How many facilities do not charge a fee to members? */

    /* Output = 4 */
    SELECT COUNT(*) AS nofeemembers
    FROM Facilities
    WHERE membercost = 0

    /* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
    where the fee is less than 20% of the facility's monthly maintenance cost.
    Return the facid, facility name, member cost, and monthly maintenance of the
    facilities in question. */

    SELECT facid, name, membercost, monthlymaintenance
    FROM Facilities
    WHERE membercost >0
    AND membercost < ( monthlymaintenance * 0.20 )

    /* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
    Try writing the query without using the OR operator. */

    SELECT *
    FROM Facilities
    WHERE facid
    IN ( 1, 5 )

    /* Q5: Produce a list of facilities, with each labelled as
    'cheap' or 'expensive', depending on if their monthly maintenance cost is
    more than $100. Return the name and monthly maintenance of the facilities
    in question. */

    SELECT name, 
    CASE WHEN monthlymaintenance >100 THEN 'expensive'
    ELSE 'cheap' END AS monthlymaintenance
    FROM Facilities

    /* Q6: You'd like to get the first and last name of the last member(s)
    who signed up. Try not to use the LIMIT clause for your solution. */

    SELECT firstname, surname
    FROM Members
    GROUP BY joindate
    ORDER BY joindate DESC
    LIMIT 1

    /* Q7: Produce a list of all members who have used a tennis court.
    Include in your output the name of the court, and the name of the member
    formatted as a single column. Ensure no duplicate data, and order by
    the member name. */

    SELECT DISTINCT CONCAT( f.name, ',', m.firstname, ' ', m.surname ) AS courtandname
    FROM Bookings
    LEFT JOIN Members AS m
    USING ( memid )
    LEFT JOIN Facilities AS f
    USING ( facid )
    WHERE f.name LIKE '%Tennis Court%'

    /* Q8: Produce a list of bookings on the day of 2012-09-14 which
    will cost the member (or guest) more than $30. Remember that guests have
    different costs to members (the listed costs are per half-hour 'slot'), and
    the guest user's ID is always 0. Include in your output the name of the
    facility, the name of the member formatted as a single column, and the cost.
    Order by descending cost, and do not use any subqueries. */

    SELECT
    CASE WHEN (f.membercost * b.slots) >30 AND memid !=0
    THEN CONCAT( f.name, ', ', m.firstname, ' ', m.surname, ', ', (f.membercost * b.slots) )
    WHEN (f.guestcost * b.slots) >30 AND memid =0
    THEN CONCAT( f.name, ', ', m.firstname, ' ', m.surname, ', ', (f.guestcost * b.slots) )
    END AS booking_cost 
    FROM Bookings AS b
    LEFT JOIN Members AS m USING ( memid )
    LEFT JOIN Facilities AS f USING ( facid )
    WHERE starttime LIKE "2012-09-14%"
    HAVING booking_cost  IS NOT NULL

    /* Q9: This time, produce the same result as in Q8, but using a subquery. */

    SELECT subq.booking_cost
    FROM (
        SELECT
        CASE WHEN (f.membercost * b.slots) >30 AND memid !=0
        THEN CONCAT( f.name, ', ', m.firstname, ' ', m.surname, ', ', (f.membercost * b.slots) )
        WHEN (f.guestcost * b.slots) >30 AND memid =0
        THEN CONCAT( f.name, ', ', m.firstname, ' ', m.surname, ', ', (f.guestcost * b.slots) )
        END AS booking_cost
        FROM Bookings AS b
        LEFT JOIN Members AS m
        USING ( memid )
        LEFT JOIN Facilities AS f
        USING ( facid )
        WHERE starttime LIKE "2012-09-14%"
        ) AS subq
    WHERE booking_cost IS NOT NULL

    /* PART 2: SQLite

    Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
    for the following questions.  

    QUESTIONS:
    /* Q10: Produce a list of facilities with a total revenue less than 1000.
    The output of facility name and total revenue, sorted by revenue. Remember
    that there's a different cost for guests and members! */

    /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


    /* Q12: Find the facilities with their usage by member, but not guests */


    /* Q13: Find the facilities usage by month, but not guests */



### Second Part using JupyterNotebook

In [1]:
# Import the necessary packages
from sqlalchemy import create_engine
import pandas as pd

# create a function to load a dataframe
def create_query(query):
    '''
    Create a database connection using sqlalchemy, and load a query into a pandas DataFrame.
    '''
    # Create an engine with the "data.sqlite" file, which was obtained by converting the output file (SB_DB.sql) from phpmyadmin
    # using the website 'https://www.rebasedata.com/conversion/9b685e05b28c23cd0f972cb42c70e28a'
    engine = create_engine('sqlite:///data.sqlite')
    con = engine.connect()
    
    # assign the 'query' variable to the connection.
    rs = con.execute(query)
    df = pd.DataFrame(rs, columns=rs.keys())
    
    # return the dataframe as 'df'
    return df

In [2]:
create_query('SELECT bookid, facid, memid, surname, firstname, recommendedby, name, membercost, guestcost, slots \
             FROM Bookings \
             LEFT JOIN Members \
             USING(memid) \
             LEFT JOIN Facilities \
             USING(facid) \
             ')

,bookid,facid,memid,surname,firstname,recommendedby,name,membercost,guestcost,slots
0,0,3,1,Smith,Darren,,Table Tennis,0.0,5.0,2
1,1,4,1,Smith,Darren,,Massage Room 1,9.9,80.0,2
2,2,6,0,GUEST,GUEST,,Squash Court,3.5,17.5,2
3,3,7,1,Smith,Darren,,Snooker Table,0.0,5.0,2
4,4,8,1,Smith,Darren,,Pool Table,0.0,5.0,1
...,...,...,...,...,...,...,...,...,...,...
4038,4038,8,29,Worthington-Smyth,Henry,2,Pool Table,0.0,5.0,2
4039,4039,8,29,Worthington-Smyth,Henry,2,Pool Table,0.0,5.0,1
4040,4040,8,21,Mackenzie,Anna,1,Pool Table,0.0,5.0,1
4041,4041,8,16,Baker,Timothy,13,Pool Table,0.0,5.0,1


In [3]:
# /* Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members! */

create_query("SELECT name, \
             SUM(CASE WHEN memid != '0' THEN (membercost * slots) \
             WHEN memid = '0' THEN (guestcost * slots) \
             END) AS total_revenue \
             FROM Bookings \
             LEFT JOIN Facilities \
             USING(facid) \
             GROUP BY name\
             HAVING total_revenue < 1000 \
             ORDER BY total_revenue DESC \
             ")

,name,total_revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0


In [4]:
# /* Q11: Produce a report of members who recommended them in alphabetic surname,firstname order */

create_query("WITH membersonly AS (SELECT * FROM Members WHERE memid != '0') \
            SELECT (m.surname || ', ' || m.firstname) AS member, (r.surname || ', ' || r.firstname) as recommendedby_name\
            FROM Members AS m \
            LEFT JOIN Members AS r ON m.recommendedby = r.memid \
            WHERE m.recommendedby <> '' \
            ORDER BY m.surname \
             ")

,member,recommendedby_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


In [5]:
# /* Q12: Find the facilities with their usage by member, but not guests */

create_query("SELECT f.name, count(b.bookid) AS member_bookings \
             FROM Facilities as f \
             INNER JOIN \
             Bookings as b ON b.facid = f.facid \
             WHERE b.memid <> 0 \
             GROUP BY f.name \
             ")

,name,member_bookings
0,Badminton Court,383
1,Massage Room 1,629
2,Massage Room 2,111
3,Pool Table,836
4,Snooker Table,444
5,Squash Court,440
6,Table Tennis,403
7,Tennis Court 1,408
8,Tennis Court 2,389


In [6]:
# /* Q13: Find the facilities usage by month, but not guests */
# Number of members, number of slots, % of facility being used
# by month, by facility

create_query("SELECT f.name, strftime('%m', b.starttime) AS month, count(b.bookid) AS member_bookings \
            FROM Facilities as f \
            INNER JOIN Bookings as b ON b.facid = f.facid \
            WHERE b.memid <> 0 \
            GROUP BY month,f.name \
            ORDER BY f.name,month \
            ")

,name,month,member_bookings
0,Badminton Court,07,56
1,Badminton Court,08,146
2,Badminton Court,09,181
3,Massage Room 1,07,123
4,Massage Room 1,08,224
5,Massage Room 1,09,282
6,Massage Room 2,07,12
7,Massage Room 2,08,40
8,Massage Room 2,09,59
9,Pool Table,07,110


---

# FOR REFERENCE

In [7]:
facs = create_query('Select * From Facilities')
facs

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [8]:
mems = create_query('SELECT * FROM Members')
mems

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
5,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
6,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,,2012-07-15 08:52:55
7,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
8,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
9,9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05


In [9]:
books = create_query('SELECT * FROM Bookings')
books

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
...,...,...,...,...,...
4038,4038,8,29,2012-09-30 16:30:00,2
4039,4039,8,29,2012-09-30 18:00:00,1
4040,4040,8,21,2012-09-30 18:30:00,1
4041,4041,8,16,2012-09-30 19:00:00,1
